# Notebook 03：03_feature_extraction.ipynb

**This is the core of motion detection**

- Extract feature vectors from keypoints
- Normalize coordinates
- Extract angles, distances
- Final feature vector usable for ML models (SVM / LSTM)


Import

In [ ]:
import numpy as np
import cv2
from ultralytics import YOLO


Load YOLO Pose Model

In [ ]:
model = YOLO("yolov8n-pose.pt")


Feature Extraction Function

In [ ]:
def extract_feature_vector(keypoints):
    """
    Convert Nx3 keypoint array into a feature vector.
    keypoints = [x, y, confidence]
    
    Features include:
    - normalized coordinates
    - distances to center
    - angles relative to center
    """

    keypoints = np.array(keypoints)

    xs = keypoints[:, 0]
    ys = keypoints[:, 1]

    # Normalize: subtract centroid
    cx, cy = np.mean(xs), np.mean(ys)
    xs_norm = xs - cx
    ys_norm = ys - cy

    # Distance from centroid
    dists = np.sqrt(xs_norm**2 + ys_norm**2)

    # Angles relative to centroid
    angles = np.arctan2(ys_norm, xs_norm)

    # Final feature vector
    return np.concatenate([xs_norm, ys_norm, dists, angles])


Example: extract featre from live camera

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)[0]

    if results.keypoints is not None:
        xy = results.keypoints.xy[0].cpu().numpy()
        conf = results.keypoints.conf[0].cpu().numpy()
        keypoints = np.hstack([xy, conf.reshape(-1, 1)])

        fv = extract_feature_vector(keypoints)
        print("Feature vector shape:", fv.shape)

    cv2.imshow("Feature Extraction", results.plot())

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
